In [1]:
# 연습문제 1. MNIST 데이터를 사용한 투표 기반 분류기
# A) 훈련 세트(50,000개), 검증 세트(10,000개), 테스트 세트(10,000개)로 나누기
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist = fetch_openml('mnist_784', version=1)

X_train_val, X_test, y_train_val, y_test = train_test_split(mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

In [2]:
print(X_train.shape)
print(y_train.shape)

print(X_val.shape)
print(y_val.shape)

print(X_test.shape)
print(y_test.shape)

(50000, 784)
(50000,)
(10000, 784)
(10000,)
(10000, 784)
(10000,)


In [5]:
# B) 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 같은 여러 종류의 분류기를 훈련
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

random_forest_clf = RandomForestClassifier(n_estimators=10, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=10, random_state=42)
# svm_clf = LinearSVC(max_iter=10000, random_state=42)
sgd_clf = SGDClassifier(random_state=42)

In [6]:
estimators = [random_forest_clf, extra_trees_clf, sgd_clf] # svm_clf,]

for estimator in estimators:
    print("훈련 예측기: ", estimator)
    estimator.fit(X_train, y_train)

훈련 예측기:  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
훈련 예측기:  ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
              

In [7]:
[estimator.score(X_val, y_val) for estimator in estimators]

[0.9469, 0.9492, 0.879]

In [8]:
# 선형 SVM의 성능이 다른 분류기보다 성능이 떨어지지만, 투표 기반 분류기의 성능을 향상시킬 수 있으므로 그대로 둔다

In [9]:
# C) 검증 세트에서 각각의 분류기보다 더 높은 성능을 내도록 이들을 직접 또는 간접 투표 분류기를 사용하는 앙상블로 연결
from sklearn.ensemble import VotingClassifier

named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_tress_clf", extra_trees_clf),
#     ("svm_clf", svm_clf),
    ("sgd_clf", sgd_clf),
]

In [10]:
voting_clf = VotingClassifier(named_estimators)

In [11]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_lea

In [12]:
voting_clf.score(X_val, y_val)

0.9529

In [13]:
[estimator.score(X_val, y_val) for estimator in voting_clf.estimators_]

[0.0, 0.0, 0.0]

In [15]:
# SVM 모델을 제거해서 성능이 향상되는지 확인해보기
# set_params()를 사용하여 None으로 지정하면 특정 예측기는 제외 가능
voting_clf.set_params(sgd_clf=None)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_lea

In [16]:
# 예측기 목록은 업데이트 됨
voting_clf.estimators

[('random_forest_clf',
  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                         criterion='gini', max_depth=None, max_features='auto',
                         max_leaf_nodes=None, max_samples=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, n_estimators=10,
                         n_jobs=None, oob_score=False, random_state=42, verbose=0,
                         warm_start=False)),
 ('extra_tress_clf',
  ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fractio

In [17]:
# 훈련된 예측기 목록은 업데이트 되지 않음 -> 다시 훈련시키거나 훈련된 예측기 목록에서 SVM 모델 제거
voting_clf.estimators_

[RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=10,
                        n_jobs=None, oob_score=False, random_state=42, verbose=0,
                        warm_start=False),
 ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=None, max_features='auto',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                 

In [18]:
# 훈련된 예측기 목록에서 SVM 모델 제거
del voting_clf.estimators_[2]

In [19]:
voting_clf.score(X_val, y_val)

0.9445

In [20]:
# 성능이 높아짐을 확인 -> SVM 모델이 성능을 저하시키는 것을 알 수 있음

In [21]:
# 간접 투표 분류기
voting_clf.voting = "soft"

In [22]:
voting_clf.score(X_val, y_val)

0.9595

In [23]:
# 성능이 많이 나아지고, 각각의 분류기보다 훨씬 좋음

In [24]:
# D) 테스트 세트로 각각의 분류기와 앙상블 분류기 성능 확인
voting_clf.score(X_test, y_test)

0.961

In [25]:
[estimator.score(X_test, y_test) for estimator in voting_clf.estimators_]

[0.0, 0.0]

In [27]:
# 연습문제 2. 스태킹 앙상블
# 위의 연습문제의 각 분류기를 실행해서 검증 세트에서 예측을 만들고, 그 결과로 새로운 훈련 세트 생성
# 각 훈련 샘플은 하나의 이미지에 대한 전체 분류기의 예측을 담은 벡터고 타깃은 이미지 클래스이다.
# 새로운 훈련 세트에 분류기 하나를 훈련시키자.
import numpy as np
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [28]:
X_val_predictions

array([[5., 5., 5.],
       [8., 8., 8.],
       [2., 2., 2.],
       ...,
       [7., 7., 7.],
       [6., 6., 6.],
       [7., 7., 7.]], dtype=float32)

In [31]:
from sklearn.ensemble import RandomForestClassifier

rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [32]:
rnd_forest_blender.oob_score_

0.9521

In [33]:
# 위의 블렌더를 더 세밀하게 튜닝하거나 다른 종류의 블렌더를 시도할 수 있음
# 교차 검증을 사용해 더 좋은 것을 선택
# 이 분류기를 모아서 스태킹 앙상블을 구성 -> 테스트 세트에 앙상블을 평가
# 테스트 세트의 각 이미지에 대해 모든 분류기로 예측을 만들고 앙상블의 예측 결과를 만들기 위해 블렌더에 그 예측을 주입
# 앞서 만든 투표 분류기와 비교

In [34]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [35]:
y_pred = rnd_forest_blender.predict(X_test_predictions)

In [36]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.9484

In [38]:
# 위 스태킹 앙상블은 간접 분류기만큼 성능을 내지 못하지만, 개개의 분류기보다는 성능이 좋음

In [2]:
# Decision Tree

# True / False 질문을 이어나가면서 학습
# 만약 tree를 만들 때 모든 자식 노드가 pure 노드가 될 때까지 진행하면 모델의 복잡도는 높아지고 과대적합이 됨

# Tree에서 과대적합을 막는 방법
# 1. tree 생성을 사전에 중단: pre-pruning
# 2. 데이터 포인트가 적은 node를 삭제하거나 병합: post_pruning

In [3]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

wine = load_wine()

x_train, x_test, y_train, y_test = train_test_split(wine.data, 
                                                    wine.target,
                                                    stratify=wine.target, 
                                                    random_state=0)

tree = DecisionTreeClassifier(random_state=0)
tree.fit(x_train, y_train)

score_tr = tree.score(x_train, y_train)
score_te = tree.score(x_test, y_test)

print('{:.3f}'.format(score_tr))
print('{:.3f}'.format(score_te))

1.000
0.889


In [5]:
# 모든 자식 노드가 pure node이므로 train set의 정확도는 100%가 됨
# tree는 train set의 모든 포인트들을 완벽하게 구분할 만큼 깊게 만들어짐 -> 과대적합의 가능성이 높다.

# 과대적합 방지 1. pre-pruning 중 하나: 최대 깊이 설정
tree = DecisionTreeClassifier(max_depth=2, random_state=0)
tree.fit(x_train, y_train)

score_tr = tree.score(x_train, y_train)
score_ts = tree.score(x_test, y_test)

print('{:.3f}'.format(score_tr))
print('{:.3f}'.format(score_ts))

0.910
0.756


In [6]:
# 훈련 데이터의 점수가 낮아진 것으로 보아 과대적합은 감소했지만 테스트 세트도 점수가 낮아졌으므로
# 이 데이터셋에 트리 모델은 적합하지 않음

In [8]:
# tree module의 export_graphviz함수를 이용해 tree를 시각화
import graphviz
from sklearn.tree import export_graphviz

xport_graphviz(tree, out_file='tree.dot',
                class_names=wine.target_names,
                feature_names=wine.feature_names,
                impurity=False, # gini 미출력
                filled=True) # filled: node의 색깔을 다르게

with open('tree.dot') as file_reader:
    dot_graph = file_reader.read()

dot = graphviz.Source(dot_graph) # dot_graph의 source 저장
dot.render(filename='tree.png') # png로 저장

NameError: name 'xport_graphviz' is not defined

In [40]:
# Random Forest

In [ ]:
# Gradient Boosting Model